## Downloading ERA5-Land hourly data at a single location (coordinate) using GEE
This notebook demonstrates how to sample raw ERA5-Land data from Google Earth Engine. Eventual improvments will tidy this data up for use in ELM. 
There is a step involved here where you must wait for GEE's computers to crank through the request, then you'll need to move a file from your Google Drive to your local machine. There are no great ways to automate this, sorry.

In [1]:
import ee
from ngeegee import e5l_utils as eu

# Make sure to Initialize with the correct project name (do not use mine--it won't work for you)
ee.Initialize(project='ee-jonschwenk')

# Define a point we want to sample
lat, lon = 68.62758, -149.59429

### Set up our request via a dictionary of parameters

In [2]:
params = {
    "gee_ic": "ECMWF/ERA5_LAND/HOURLY", # don't change this
    "start_date": "1950-01-01", 
    "end_date": "2100-01-05", # If your end date is longer than what's available, it will just truncate at the last available date. Here I've used the year 2100 to ensure we download all data.
    "gee_bands": ["temperature_2m", 
                  "total_precipitation",
                  'dewpoint_temperature_2m',
                  'snow_albedo'], # select the bands (variables) you want to sample. Must exactly match the band names of the ERA5_LAND/HOURLY imageCollection
    "point": (lon, lat), 
    "gdrive_folder": "NGEE_test",  # Google Drive folder name - will be created if it doesn't exist
    "filename": "ngee_test_era5_timeseries.csv"  # Output CSV file name
}

# Send the job to GEE!
eu.sample_e5lh_at_point(params)

'Export task started: ngee_test_era5_timeseries.csv (Check Google Drive or Task Status in the Javascript Editor)'

### Now we wait.
You've sent a Task to Google Earth Engine. You can check on its state using the [GEE Javascript code editor](http://code.earthengine.google.com) by clicking the `Tasks` tab in the upper-right panel.
Eventually it will finish, and your csv will show up where you told GEE to put it: `gdrive_folder/filename`.
It should not take more than a few hours to fetch all variables (37 I think) for all timesteps. In my testing here, it took 37 minutes to fetch two variables from 2000-2025.
Once that file is created in your GDrive, download it to your local machine and hammer away!